In [4]:
import numpy as np

In [5]:
def simple_pairwise_sum_python(arr1, arr2):
    npts1, npts2 = len(arr1), len(arr2)
    result = np.zeros(npts1*npts2)

    for i in range(0, npts1):
        x = arr1[i]

        for j in range(npts2):
            y = arr2[j]

            idx_result = i*npts2 + j
            result[idx_result] = x + y

    return result

In [6]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [7]:
%%cython
import numpy as np # use import to get numpy functions
cimport numpy as cnp # use cimport to get numpy types, naming as cnp for clarity

cimport cython # only necessary for the performance-enhancing decorators

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def simple_pairwise_sum_cython(arr1, arr2):

    cdef int npts1 = len(arr1)
    cdef int npts2 = len(arr2)

    cdef cnp.float64_t[:] result = np.zeros(npts1*npts2, dtype=np.float64)

    cdef cnp.float64_t[:] arr1_view = np.ascontiguousarray(arr1, dtype=np.float64)
    cdef cnp.float64_t[:] arr2_view = np.ascontiguousarray(arr2, dtype=np.float64)

    cdef int i, j, idx_result
    cdef cnp.float64_t x, y

    for i in range(0, npts1):
        x = arr1_view[i]

        for j in range(npts2):
            y = arr2_view[j]

            idx_result = i*npts2 + j
            result[idx_result] = x + y

    return np.array(result)

In [8]:
npts = int(1e3)
arr1 = np.random.random(npts)
arr2 = np.random.random(npts)

In [9]:
%timeit simple_pairwise_sum_python(arr1, arr2)

1 loop, best of 3: 512 ms per loop


In [10]:
%timeit simple_pairwise_sum_cython(arr1, arr2)

100 loops, best of 3: 9.02 ms per loop
